In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle

/Users/atalmanuja/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer,HashingVectorizer
from sklearn import cross_validation
from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier,ExtraTreesClassifier,AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
import unicodedata
import os
from keras.preprocessing import sequence

/Users/atalmanuja/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [3]:
text=pd.read_csv('mbti_1.csv')
print(text.shape)
print(text[0:5])
print(text.iloc[2])

(8675, 2)
   type                                              posts
0  INFJ  'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1  ENTP  'I'm finding the lack of me in these posts ver...
2  INTP  'Good one  _____   https://www.youtube.com/wat...
3  INTJ  'Dear INTP,   I enjoyed our conversation the o...
4  ENTJ  'You're fired.|||That's another silly misconce...
type                                                  INTP
posts    'Good one  _____   https://www.youtube.com/wat...
Name: 2, dtype: object


8675

50000

In [4]:
posts = text.posts
labels = text.type
full_posts =[]
full_labels = []
for i in range(len(posts)):
    temp = posts[i].split('|||')
    for j in temp:
        full_posts.append(j)
        full_labels.append(labels[i])
d = {'posts':full_posts, 'type':full_labels}
text = pd.DataFrame(d)
n = len(text)
text = text[:n] 
full_labels = full_labels[:n]
full_posts = full_posts[:n]

In [6]:
len(text)

50000

In [7]:
text.head()

,posts,type
0,'http://www.youtube.com/watch?v=qsXHcwe3krw,INFJ
1,http://41.media.tumblr.com/tumblr_lfouy03PMA1q...,INFJ
2,enfp and intj moments https://www.youtube.com...,INFJ
3,What has been the most life-changing experienc...,INFJ
4,http://www.youtube.com/watch?v=vXZeYwwRDw8 h...,INFJ


In [8]:
'''
I = 0 , E = 1
S = 0 , N = 1
F = 0 , T = 1
J = 0 , P = 1
'''



'\nI = 0 , E = 1\nS = 0 , N = 1\nF = 0 , T = 1\nJ = 0 , P = 1\n'

In [9]:
personalities = {'ISFJ':[0,0,0,0],
                'ISFP':[0,0,0,1],
                'ISTJ':[0,0,1,0],
                'ISTP':[0,0,1,1],
                'INFJ':[0,1,0,0],
                'INFP':[0,1,0,1],
                'INTJ':[0,1,1,0],
                'INTP':[0,1,1,1],
                'ESFJ':[1,0,0,0],
                'ESFP':[1,0,0,1],
                'ESTJ':[1,0,1,0],
                'ESTP':[1,0,1,1],
                'ENFJ':[1,1,0,0],
                'ENFP':[1,1,0,1],
                'ENTJ':[1,1,1,0],
                'ENTP':[1,1,1,1]}

In [10]:
full_labels[0]

'INFJ'

In [11]:
for i in range(len(full_labels)):
    full_labels[i] = personalities[full_labels[i]]

In [12]:
text = {'posts':full_posts, 'type':full_labels}
text =pd.DataFrame(text)

In [13]:
len(text)

50000

In [14]:
'''
from sklearn.preprocessing import LabelBinarizer

# One hot encode labels
labels=text.type
encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)
labels=encoder.fit_transform(labels)
labels=np.array(labels)
print(labels[50:55])
'''

'\nfrom sklearn.preprocessing import LabelBinarizer\n\n# One hot encode labels\nlabels=text.type\nencoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)\nlabels=encoder.fit_transform(labels)\nlabels=np.array(labels)\nprint(labels[50:55])\n'

In [15]:
sequence_length = 80

In [16]:
import numpy as np
import re
import itertools
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
"""
Original taken from https://github.com/dennybritz/cnn-text-classification-tf
"""

def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    #print string
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", "", string)
    string = re.sub(r"\)", "", string)
    string = re.sub(r"\?", "", string)
    string = re.sub(r"/", "", string)
    string = re.sub(r"\s{2,}", " ", string)
    #print (string)
    return string.strip().lower()


def load_data_and_labels():
    """
    Loads MR polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    # Load data from files
    # Split by words
    #x_text = data['utterance'].as_matrix()
    x_text = text.posts.as_matrix()
    '''for i in range(len(x_text)):
        if type(x_text[i])==unicode:
            x_text[i]=unicodedata.normalize('NFKD', x_text[i]).encode('ascii','ignore')'''
    x_text = [clean_str(sent) for sent in x_text]
    #print x_text[0:2]
    x_text = [s.split(" ") for s in x_text]
    
    # Generate labels
    y=labels
#     global encoder
#     encoder = LabelEncoder()
#     encoder.fit(y)
#     encoded_Y = encoder.transform(y)
#     dummy_y=np_utils.to_categorical(encoded_Y)
    #print x_text
    return [x_text, y]


def pad_sentences(sentences, padding_word="<PAD/>"):
    """
    Pads all sentences to the same length. The length is defined by the longest sentence.
    Returns padded sentences.
    """
#     sequence_length = max(len(x) for x in sentences)
#     print (min(len(x) for x in sentences))
#     sequence_length+=1
    #print (sequence_length)
    #sequence_length=20
    padded_sentences = []
    for i in range(len(sentences)):
        sentence = sentences[i]
        num_padding = sequence_length - len(sentence)
        #print (sentence)
        if num_padding<=0:
            temp=sentence[0:sequence_length-1]
            temp.append(padding_word)
            new_sentence=temp
        else:
            new_sentence = sentence + [padding_word] * num_padding
        
        padded_sentences.append(new_sentence)
    #print (padded_sentences[0],len(padded_sentences[0]))
    return padded_sentences


def build_vocab(sentences):
    """
    Builds a vocabulary mapping from word to index based on the sentences.
    Returns vocabulary mapping and inverse vocabulary mapping.
    """
    # Build vocabulary
    word_counts = Counter(itertools.chain(*sentences))
    # Mapping from index to word
    #print (word_counts)
    #print (word_counts.most_common())
    vocabulary_inv = [x[0] for x in word_counts.most_common()]
    # Mapping from word to index
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    return [vocabulary, vocabulary_inv]


def build_input_data(sentences, labels, vocabulary):
    """
    Maps sentencs and labels to vectors based on a vocabulary.
    """
    '''for sentence in sentences:
        #print type(sentences)
        print ('*******************************************')
    #    print sentences[0:2]
        for word in sentence:
            print type(word)
    print sentences'''
    x = np.array([[vocabulary[str(word)] for word in sentence] for sentence in sentences])
    print (x.dtype)
    y = np.array(labels)
    return [x, y]


def load_data():
    """
    Loads and preprocessed data for the MR dataset.
    Returns input vectors, labels, vocabulary, and inverse vocabulary.
    """
    # Load and preprocess data
    sentences, labels = load_data_and_labels()
    sentences = np.array(sentences)
    #print sentences[0:2]
    sentences_padded = pad_sentences(sentences)
    vocabulary, vocabulary_inv = build_vocab(sentences_padded)
    #print ('vocab_inv')
    #print (vocabulary_inv)
    #print ('vocab')
    #print (vocabulary)
    x, y = build_input_data(sentences_padded, labels, vocabulary)
    return [x, y, vocabulary, vocabulary_inv]


In [17]:
sentences, labels = load_data_and_labels()

In [18]:
type(sentences)

list

In [19]:
lebels = full_labels

In [20]:
sentences

[["'http", 'www', 'youtube', 'com', 'watchv', 'qsxhcwe3krw'],
 ['http',
  '41',
  'media',
  'tumblr',
  'com',
  'tumblr',
  'lfouy03pma1qa1rooo1',
  '500',
  'jpg'],
 ['enfp',
  'and',
  'intj',
  'moments',
  'https',
  'www',
  'youtube',
  'com',
  'watchv',
  'iz7le1g4xm4',
  'sportscenter',
  'not',
  'top',
  'ten',
  'plays',
  'https',
  'www',
  'youtube',
  'com',
  'watchv',
  'ucdfze1etec',
  'pranks'],
 ['what',
  'has',
  'been',
  'the',
  'most',
  'life',
  'changing',
  'experience',
  'in',
  'your',
  'life'],
 ['http',
  'www',
  'youtube',
  'com',
  'watchv',
  'vxzeywwrdw8',
  'http',
  'www',
  'youtube',
  'com',
  'watchv',
  'u8ejam5dp3e',
  'on',
  'repeat',
  'for',
  'most',
  'of',
  'today'],
 ['may', 'the', 'perc', 'experience', 'immerse', 'you'],
 ['the',
  'last',
  'thing',
  'my',
  'infj',
  'friend',
  'posted',
  'on',
  'his',
  'facebook',
  'before',
  'committing',
  'suicide',
  'the',
  'next',
  'day',
  'rest',
  'in',
  'peace',
  'ht

In [21]:
sentences_padded = pad_sentences(sentences)

In [22]:
vocabulary, vocabulary_inv = build_vocab(sentences_padded)


In [23]:
x, y = build_input_data(sentences_padded, labels, vocabulary)

int64


In [24]:
y = full_labels

In [25]:
len(y)

50000

In [26]:
embedding_dim = 300         


In [27]:
from gensim.models import KeyedVectors
wvmodel = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)


In [28]:
def getEmbedings(vocabulary_inv):
    embedding_weights = [np.array([wvmodel[w] if w in wvmodel\
                                                        else np.random.uniform(-0.25,0.25,wvmodel.vector_size)\
                                                        for w in vocabulary_inv])]
    return embedding_weights


embedding_weights=getEmbedings(vocabulary_inv)

In [29]:
x = embedding_weights[0][x]

In [30]:
x.shape

(50000, 80, 300)

In [31]:
a = y

In [32]:
a = np.array(a)

In [33]:
a.shape

(50000, 4)

In [34]:
from keras.layers import LSTM
from keras import optimizers
from keras.models import Sequential, Model
from keras.models import Sequential
from keras.layers import Dense
import numpy
import pandas as pd
from keras.utils import np_utils
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Input, Merge, Convolution1D, MaxPooling1D
from keras.regularizers import L1L2
from keras.layers.normalization import BatchNormalization


In [35]:
model = Sequential()
model.add(Dropout(0.25, input_shape=(sequence_length, embedding_dim)))
#model.add(Embedding(len(vocabulary), embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(200,activation='tanh'))
model.add(BatchNormalization())
#model.add(Dropout(0.015))
#model.add(LSTM(200,activation='tanh'))
# model.add(Dropout(0.005))
# model.add(LSTM(100,activation='relu',return_sequences=True))
# model.add(LSTM(50))
model.add(Dense(4))
model.add(Activation('sigmoid'))

#rmsprop=optimizers.rmsprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_1 (Dropout)          (None, 80, 300)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 200)               400800    
_________________________________________________________________
batch_normalization_1 (Batch (None, 200)               800       
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 804       
_________________________________________________________________
activation_1 (Activation)    (None, 4)                 0         
Total params: 402,404
Trainable params: 402,004
Non-trainable params: 400
_________________________________________________________________
None


In [36]:
cfl = model.fit(x, a, epochs=10, validation_split=0.2,batch_size=256)

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 904s 23ms/step - loss: 2.8420 - acc: 0.5801 - val_loss: 2.7936 - val_acc: 0.7050
Epoch 2/10
40000/40000 [==============================] - 837s 21ms/step - loss: 2.7785 - acc: 0.6804 - val_loss: 2.7921 - val_acc: 0.7050
Epoch 3/10
40000/40000 [==============================] - 820s 20ms/step - loss: 2.7750 - acc: 0.6804 - val_loss: 2.7921 - val_acc: 0.7050
Epoch 4/10
35328/40000 [=========================>....] - ETA: 1:25 - loss: 2.7707 - acc: 0.6808

KeyboardInterrupt: 

In [ ]:
print(cfl.history.keys())

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline 
fig = plt.gcf()
plt.plot(cfl.history['acc'])
plt.plot(cfl.history['val_acc'])
plt.title("Accuracy Curve")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(['train','test'], loc="upper left")
plt.show()
fig.savefig("Model_Accuracy_Curves.png")

NameError: name 'cfl' is not defined

In [65]:
model.save('version3.h5')

In [37]:
from keras.models import load_model
model = load_model('version3.h5')

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_1 (Dropout)          (None, 80, 300)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 200)               400800    
_________________________________________________________________
batch_normalization_1 (Batch (None, 200)               800       
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 804       
_________________________________________________________________
activation_1 (Activation)    (None, 4)                 0         
Total params: 402,404
Trainable params: 402,004
Non-trainable params: 400
_________________________________________________________________


In [38]:
query="What has been the most life-changing experience in your life?"
x_=pad_sentences(sentences)
x = np.array([[vocabulary[word] if word in vocabulary.keys() else 0 for word in sentence.split()] for sentence in [query]])
print (x)

x=embedding_weights[0][x]
print ((model.predict_classes(x)))
print (model.predict_proba(x))

80
["'http", 'www', 'youtube', 'com', 'watchv', 'qsxhcwe3krw', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>'] 80
[[  0  96  76   3  84   0 299  12  42   0]]


ValueError: Error when checking : expected dropout_1_input to have shape (None, 80, 300) but got array with shape (1, 10, 300)

In [16]:
text.posts[3]

'What has been the most life-changing experience in your life?'

In [70]:
query="What has been the most life-changing experience in your life?"
query = query.split()

In [71]:
query

['What',
 'has',
 'been',
 'the',
 'most',
 'life-changing',
 'experience',
 'in',
 'your',
 'life?']

In [104]:
x_=pad_sentences([query])


80
['What', 'has', 'been', 'the', 'most', 'life-changing', 'experience', 'in', 'your', 'life?', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>', '<PAD/>'] 80


In [109]:
x = []
keys = list(vocabulary.keys())
for sentence in x_:
    for word in sentence:
        if word in keys:
            x.append(vocabulary[word])
        else:
            x.append(0)

In [123]:
x = np.array(x)

In [124]:
x=embedding_weights[0][x]

IndexError: arrays used as indices must be of integer (or boolean) type

In [126]:
x.shape

(1, 80, 300)

In [127]:
x

array([[[-0.12835245,  0.08800022, -0.07651804, ..., -0.00952924,
         -0.07180631, -0.04523096],
        [-0.04492188, -0.03039551,  0.00234985, ...,  0.01226807,
         -0.08837891, -0.015625  ],
        [-0.10107422,  0.0177002 ,  0.01470947, ..., -0.08349609,
          0.00558472, -0.01782227],
        ..., 
        [-0.12835245,  0.08800022, -0.07651804, ..., -0.00952924,
         -0.07180631, -0.04523096],
        [-0.12835245,  0.08800022, -0.07651804, ..., -0.00952924,
         -0.07180631, -0.04523096],
        [-0.12835245,  0.08800022, -0.07651804, ..., -0.00952924,
         -0.07180631, -0.04523096]]])

In [128]:
model.predict(x)

array([[ 0.02559696,  0.94552737,  0.75336689,  0.74820584]], dtype=float32)

In [62]:
query = 'what the hell bro'

In [63]:
def predict_custom(query):
    query = clean_str(query)
    query = query.split()
    x_=pad_sentences([query])
    x = []
    keys = list(vocabulary.keys())
    for sentence in x_:
        for word in sentence:
            if word in keys:
                x.append(vocabulary[word])
            else:
                x.append(0)
    x = np.array([x])
    x=embedding_weights[0][x]
    return model.predict(x) 

In [64]:
predict_custom(query)

array([[ 0.19859233,  0.83251077,  0.39674428,  0.59449583]], dtype=float32)

In [169]:
query = text.posts[49970]

In [170]:
text.type[49970]

[0, 0, 1, 1]

In [174]:
s = 0
for j in text.type:
    if j ==[0,1,0,1]:
        s=s+1

In [175]:
s

15437

In [176]:
sen = 'hello how are you. i am having a great day'

In [177]:
predict_custom(sen)

array([[ 0.02557908,  0.94551784,  0.75333619,  0.74818897]], dtype=float32)

In [54]:
type(embedding_weights)

list

In [55]:
embedding_weights = np.array(embedding_weights)

In [65]:
np.save('embedding_weights.npy',embedding_weights)

dict